<a href="https://colab.research.google.com/github/serhii-anopriienko/python-sitemap-checker/blob/main/%E2%AD%90%EF%B8%8FSitemap_Tech_Audit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automate Sitemap Audit with Python

XML sitemaps are designed to make life easier for search engines by providing an index of a site’s URLs. However, they’re also a useful tool in competitor analysis and allow you to quickly identify all of a site’s pages and the level of importance the site assigns to each page.

The following Python script may help you cut off plenty of time of manual technical research and to fetch plenty of juicy data insights at the same time.

#Requirements and Assumptions

- Run the script on **Google Colab**
- Make sure to `!pip install` advertools and pandas packages
- Remember that **[sitemaps are a recommendation](https://developers.google.com/search/docs/advanced/sitemaps/build-sitemap)** to Google about which pages you think are important; Google does not pledge to crawl every URL in a sitemap.


In [ ]:
#@title Install Packages

%%capture
!pip install advertools dash dash_bootstrap_components jupyter_dash plotly bs4 matplotlib datasets adviz dash_bootstrap_templates


In [ ]:
#@title Import Packages

import os
import requests
import urllib.parse
from bs4 import BeautifulSoup

import time
import warnings

import advertools as adv
import adviz
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
from IPython.display import display_html, display_markdown, HTML
from IPython.core.display import display

from lxml import etree

import matplotlib
import sklearn

from dash_bootstrap_templates import load_figure_template
import dash
import dash_core_components as dcc
import dash_html_components as html

# Set display options
pd.options.display.max_columns = None
warnings.filterwarnings("ignore")
display(HTML("<style>.container { width:100% !important; }</style>"))

def md(text):
    return display_markdown(text, raw=True)

# Check package versions
for pkg in [adv, pd, plotly, sklearn]:
    print(f'{pkg.__name__:-<30}v{pkg.__version__}')

load_figure_template(['darkly','bootstrap','flatly','cosmo'])


<ipython-input-2-7b0a5b06c8d8>:30: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-7b0a5b06c8d8>:31: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


advertools--------------------v0.13.5
pandas------------------------v1.5.3
plotly------------------------v5.15.0
sklearn-----------------------v1.2.2


In [ ]:
#@title Fetch and Parse XML sitemaps from the Robots.TXT

sitemap = adv.sitemap_to_df('https://yoast.com/robots.txt')
                            #max_workers=8,
                           # recursive=True)
sitemap.to_csv('sitemap_collection.csv',index=False)

#parse the sitemaps

sitemap_raw = pd.read_csv('sitemap_collection.csv')#, parse_dates=['lastmod', 'download_date'], low_memory=False)

#overview duplicates
print(f'Original: {sitemap_raw.shape}')
sitemap = sitemap_raw.drop_duplicates(subset=['loc'])
print(f'After removing dupes: {sitemap.shape}')
sitemap.to_csv('sitemap.csv',index=False)
sitemap


INFO:root:Getting https://yoast.com/yoast_jobs-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_events-sitemap.xml
INFO:root:Getting https://yoast.com/page-sitemap.xml
INFO:root:Getting https://yoast.com/post-sitemap2.xml
INFO:root:Getting https://yoast.com/yoast_courses-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_employees-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_banen-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_plugins-sitemap.xml
INFO:root:Getting https://yoast.com/wpkb-article-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_webhost-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_videos-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_feature-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_care_fund-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_podcast-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_shopify-sitemap.xml
INFO:root:Getting https://yoast.com/yoast_developer_blog-sitemap.xml
INFO:root:Getting http

Original: (2462, 21)
After removing dupes: (2215, 21)


,loc,lastmod,image,image_loc,sitemap,sitemap_size_mb,download_date,video,video_title,video_publication_date,video_description,video_player_loc,video_thumbnail_loc,video_duration,video_view_count,video_width,video_height,video_family_friendly,video_uploader,video_tag,video_content_loc
0,https://yoast.com/jobs/,2024-01-08 09:39:56+00:00,NaN,NaN,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://yoast.com/jobs/retention-marketing-manager/,2023-12-22 12:18:57+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://yoast.com/jobs/seo-specialist/,2023-12-22 13:37:00+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://yoast.com/jobs/marketing-internship/,2024-01-08 09:39:56+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/12/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://yoast.com/about-us/events/,2024-01-24 10:45:35+00:00,NaN,NaN,https://yoast.com/yoast_events-sitemap.xml,0.001593,2024-01-27 18:37:01.330218+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,https://yoast.com/yoast-seo-3-2/,NaN,NaN,NaN,https://yoast.com/video-sitemap.xml,0.323858,2024-01-27 18:37:07.320142+00:00,\n\t\t\t,Yoast SEO 3.2 • Yoast,2016-04-20T11:38:57+00:00,"The Yoast SEO 3.2 release contains accessibility improvements, a new help center and, in Premium, Social Previews fo...",https://www.youtube.com/embed/w6rMqyFLhgA,https://yoast.com/app/uploads/2016/04/w6rmqyflhga.jpg,94.0,9442.0,480.0,270.0,yes,Joost de Valk,Yoast SEO,NaN
2322,https://yoast.com/yoast-seo-3-3/,NaN,NaN,NaN,https://yoast.com/video-sitemap.xml,0.323858,2024-01-27 18:37:07.320142+00:00,\n\t\t\t,Yoast SEO 3.3: content analysis • Yoast,2016-06-14T12:52:59+00:00,"We're very proud to release Yoast SEO 3.3! This is a huge update, with many new features, like our new content analy...",https://www.youtube.com/embed/Yn85c-tfbjs,https://yoast.com/app/uploads/2016/06/yn85c-tfbjs.jpg,92.0,39906.0,480.0,270.0,yes,Joost de Valk,Yoast SEO,NaN
2323,https://yoast.com/yoast-seo-9-4/,NaN,NaN,NaN,https://yoast.com/video-sitemap.xml,0.323858,2024-01-27 18:37:07.320142+00:00,\n\t\t\t,Yoast SEO 9.4: Help us beta test the new SEO analysis! • Yoast,2019-01-08T11:31:54+00:00,Yoast SEO 9.4 brings a beta version of the new SEO analysis that will land in Yoast SEO 10. Please help us test it! ...,https://www.youtube.com/embed/OZFSwc4Wp2s,https://yoast.com/app/uploads/2019/01/ozfswc4wp2s.jpg,901.0,5514.0,480.0,270.0,yes,Edwin Toonen,Yoast SEO 9.4,NaN
2331,https://yoast.com/yoast-seo-10-0/,NaN,NaN,NaN,https://yoast.com/video-sitemap.xml,0.323858,2024-01-27 18:37:07.320142+00:00,\n\t\t\t,Yoast SEO 10.0: Meet the new SEO analysis • Yoast,2019-03-12T10:45:08+00:00,Yoast SEO 10.0 is out! This release brings a new SEO analysis. Here's your chance to write even better content for r...,https://www.youtube.com/embed/OZFSwc4Wp2s,https://yoast.com/app/uploads/2019/03/ozfswc4wp2s.jpg,901.0,5514.0,480.0,270.0,yes,Edwin Toonen,Yoast SEO 10.0,NaN


#XML Sitemap Properties: Size and Number of URLs

In [ ]:
#Number of URLs - make sure they don't exceed 50k
sitemap['sitemap'].value_counts()

https://yoast.com/post-sitemap.xml                    903
https://yoast.com/wpkb-article-sitemap.xml            374
https://yoast.com/yoast_developer_blog-sitemap.xml    113
https://yoast.com/yoast_employees-sitemap.xml         113
https://yoast.com/yoast_care_fund-sitemap.xml         109
https://yoast.com/page-sitemap.xml                    103
https://yoast.com/yoast_wordpress_core-sitemap.xml     82
https://yoast.com/yoast_videos-sitemap.xml             75
https://yoast.com/post_tag-sitemap.xml                 70
https://yoast.com/yoast_webinar-sitemap.xml            70
https://yoast.com/yoast_feature-sitemap.xml            59
https://yoast.com/yoast_webhost-sitemap.xml            49
https://yoast.com/yoast_courses-sitemap.xml            18
https://yoast.com/yoast_plugins-sitemap.xml            18
https://yoast.com/yoast_podcast-sitemap.xml            18
https://yoast.com/video-sitemap.xml                    14
https://yoast.com/yoast_series-sitemap.xml              7
https://yoast.

In [ ]:
#Sitemap Size - make sure they don't exceed 50 MB = 0.5
sitemap.drop_duplicates().sort_values(by='sitemap_size_mb',ascending=False)

,loc,lastmod,image,image_loc,sitemap,sitemap_size_mb,download_date,video,video_title,video_publication_date,video_description,video_player_loc,video_thumbnail_loc,video_duration,video_view_count,video_width,video_height,video_family_friendly,video_uploader,video_tag,video_content_loc
1108,https://yoast.com/pagination-infinite-scrolling/,2019-03-22 13:35:55+00:00,\n\t\t\t,https://yoast.com/app/uploads/2016/11/wp-beginner.png,https://yoast.com/post-sitemap.xml,0.458994,2024-01-27 18:37:07.054225+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1392,https://yoast.com/rewrite-republish-article-with-yoast-duplicate-post/,2021-09-22 14:42:32+00:00,\n\t\t\t,https://yoast.com/app/uploads/2021/01/yoast-rewrite-republish-reschedule-option.jpg,https://yoast.com/post-sitemap.xml,0.458994,2024-01-27 18:37:07.054225+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1390,https://yoast.com/yoast-seo-17-2/,2021-09-22 09:58:18+00:00,\n\t\t\t,https://yoast.com/app/uploads/2021/09/yoast_seo_17.2_fi.png,https://yoast.com/post-sitemap.xml,0.458994,2024-01-27 18:37:07.054225+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1389,https://yoast.com/woocommerce-seo-14-4/,2021-09-15 15:51:42+00:00,\n\t\t\t,https://yoast.com/app/uploads/2021/09/product-specific-analysis-yoast-woocommerce-seo.jpg,https://yoast.com/post-sitemap.xml,0.458994,2024-01-27 18:37:07.054225+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1388,https://yoast.com/yoast-seo-17-1/,2021-09-07 08:42:23+00:00,\n\t\t\t,https://yoast.com/app/uploads/2021/09/product-specific-analysis-yoast-woocommerce-seo.jpg,https://yoast.com/post-sitemap.xml,0.458994,2024-01-27 18:37:07.054225+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,https://yoast.com/series/why-we-love-wordpress/,2023-07-04 12:13:32+00:00,NaN,NaN,https://yoast.com/yoast_series-sitemap.xml,0.001335,2024-01-27 18:37:06.284049+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,https://yoast.com/series/wordpress-for-beginners/,2023-12-20 13:26:50+00:00,NaN,NaN,https://yoast.com/yoast_series-sitemap.xml,0.001335,2024-01-27 18:37:06.284049+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1070,https://yoast.com/series/annelieke-analytics/,2022-12-06 15:03:40+00:00,NaN,NaN,https://yoast.com/yoast_series-sitemap.xml,0.001335,2024-01-27 18:37:06.284049+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1071,https://yoast.com/series/ask-yoast/,2023-08-23 20:58:34+00:00,NaN,NaN,https://yoast.com/yoast_series-sitemap.xml,0.001335,2024-01-27 18:37:06.284049+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Errors in Sitemaps?

In [ ]:
# are there any
'errors' in sitemap # ?

False

In [ ]:
#@title Find out which URLs have errors
sitemap[sitemap['errors'].isna()]['loc'].sample(20).tolist()

KeyError: ignored

#Scrape URLs From A Single XML Sitemap

### columns could be:
columns=['loc',	'lastmod',	'changefreq',	'priority',	'image',	'image_loc',	'image_caption',	'image_title',	'sitemap',	'sitemap_size_mb',	'download_date']

In [ ]:
x = adv.sitemap_to_df("https://yoast.com/yoast_developer_blog-sitemap.xml")
df = pd.DataFrame(x, columns=['loc', 'lastmod', 'sitemap', 'sitemap_size_mb', 'download_date'])
df['lastmod'] = df['lastmod'].dt.strftime('%Y-%m-%d')
#drop multiple columns
#sitemap_url = df.drop(['sitemap_last_modified'], axis=1)
print(len(df))
df.head()


INFO:root:Getting https://yoast.com/yoast_developer_blog-sitemap.xml


113


,loc,lastmod,sitemap,sitemap_size_mb,download_date
0,https://yoast.com/developer-blog/,2024-01-18,https://yoast.com/yoast_developer_blog-sitemap.xml,0.039176,2024-01-27 19:11:35.995357+00:00
1,https://yoast.com/developer-blog/release-candidate-yoast-seo-for-magento-2-now-available/,2017-05-05,https://yoast.com/yoast_developer_blog-sitemap.xml,0.039176,2024-01-27 19:11:35.995357+00:00
2,https://yoast.com/developer-blog/dont-rely-title-attribute/,2017-06-09,https://yoast.com/yoast_developer_blog-sitemap.xml,0.039176,2024-01-27 19:11:35.995357+00:00
3,https://yoast.com/developer-blog/accessibility-future-you/,2017-07-28,https://yoast.com/yoast_developer_blog-sitemap.xml,0.039176,2024-01-27 19:11:35.995357+00:00
4,https://yoast.com/developer-blog/x-redirect-by-header/,2017-10-27,https://yoast.com/yoast_developer_blog-sitemap.xml,0.039176,2024-01-27 19:11:35.995357+00:00


In [ ]:
#@title Inspect Publishing Trends based on the lastmod
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df['lastmod'].sort_values(),
        y=np.arange(1, len(df) + 1) / len(df),
        mode='markers',
        marker=dict(size=15, opacity=0.8, color='gold'),
        name='ECDF'
    )
)

fig.add_shape(
    type='rect',
    x0='2022-01-01', x1='2022-12-31', y0=0, y1=1,
    fillcolor='gray', opacity=0.2,
    xref='x', yref='paper',
    layer='below',
    name='No publishing',
)

fig.add_shape(
    type='rect',
    x0='2023-01-01', x1='2023-07-05', y0=0, y1=1,
    fillcolor='gray', opacity=0.2,
    xref='x', yref='paper',
    layer='below',
    name='Frequent publishing',
)

fig.update_layout(
    title='Kellogs publishing trends<br>(lastmod tags of XML sitemap)',
    template='plotly_dark',
    height=500
)

fig.show()


In [ ]:
#@title 2️⃣ Check Tag Usage Within The Sitemap (If Existing)

#How many URLs have a date in the URL

''' sitemap_last_modified could be replaced by sitemap_last_modified or the column name triggering a date
'''

md(f"## URLs that have lastmod implemented: {df['lastmod'].notna().sum():,} ({df['lastmod'].notna().mean():.1%})")
#check for other tags
md(f"## URLs that have priority implemented: {df['priority'].notna().sum():,} ({df['priority'].notna().mean():.1%})")
md(f"## URLs that have changefreq implemented: {df['changefreq'].notna().sum():,} ({df['changefreq'].notna().mean():.1%})")

## URLs that have lastmod implemented: 374 (100.0%)

KeyError: 'priority'

In [ ]:
#@title Find URLs without lastMode
sitemap[sitemap['lastmod'].isna()]['loc'].tolist()

# 📓 Sidenote
- ✅ You want to make sure you have got `<loc>` and `<lastmod>` implemented
- ❌ If you're displaying your sitemap to Googlebot, you want to [avoid using  `<priority>` and `<changefreq>`](https://https://developers.google.com/search/docs/advanced/sitemaps/build-sitemap) values as Google ignores them.

#Check if Pagination or Internal Search parameters are submitted on the XML sitemap

In [ ]:
# Check if pagination or Internal Search parameters are present in the 'loc' column of the sitemap
sitemap['Pagination'] = sitemap['loc'].str.contains(r'/p=|\\?page=|searchajax|cgid', regex=True, na=False)

if sitemap['Pagination'].any():
    print('Pagination/Internal Search parameters found')
    sitemap.to_csv('pagination.csv', index=False)
else:
    print('No Pagination')

sitemap.head()


No Pagination


,loc,lastmod,image,image_loc,sitemap,sitemap_size_mb,download_date,video,video_title,video_publication_date,video_description,video_player_loc,video_thumbnail_loc,video_duration,video_view_count,video_width,video_height,video_family_friendly,video_uploader,video_tag,video_content_loc,Pagination
0,https://yoast.com/jobs/,2024-01-08 09:39:56+00:00,NaN,NaN,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,https://yoast.com/jobs/retention-marketing-manager/,2023-12-22 12:18:57+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,https://yoast.com/jobs/seo-specialist/,2023-12-22 13:37:00+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,https://yoast.com/jobs/marketing-internship/,2024-01-08 09:39:56+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/12/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,https://yoast.com/about-us/events/,2024-01-24 10:45:35+00:00,NaN,NaN,https://yoast.com/yoast_events-sitemap.xml,0.001593,2024-01-27 18:37:01.330218+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


#Ratio of URLs to Outbound Links

This finds how many external links were found on URLs submitted on a given XML sitemap.
This will be executed by running a casual sample of 25% of pages.
The output will return no more than 40 sampled pages  

In [ ]:
#@title Indicate the Homepage of the XML sitemap you're going to audit
homepage = 'https://www.kering.com'

In [ ]:
def count_links( page_url, domain ):
    """Given input page_url, output the total number of outbound links"""
    links_internal = {}
    links_external = {}

    # download the html
    res = requests.get(page_url)
    if "html" not in res.headers.get('Content-Type'):
        # this is an image
        return {'parseable': False}

    html = res.text
    soup = BeautifulSoup(html)

    for a in soup.find_all('a'):
        link = a.get('href')

        # skip missing
        if link is None:
            continue

        # remove params
        link = link.split("?")[0]
        # remove shortcuts
        link = link.split("#")[0]

        # skip missing
        if (link is None) or (link == ''):
            continue

        if (domain in link) or (len(link)>1 and link[0:1]=="/") or (len(link)>2 and link[0:2]=="./"):
            # is internal
            links_internal[link]= links_internal.get(link,0) + 1
        else:
            # external
            links_external[link]= links_external.get(link,0) + 1

    return {"parseable": True, "external": links_external, "internal": links_internal }


domain = urllib.parse.urlparse(homepage).netloc

# test one url
page_url = sitemap.iloc[0]['loc']
print(page_url)

links = count_links( page_url, domain )
links

https://www.kering.com/it


{'parseable': True,
 'external': {'https://www.instagram.com/kering_official/': 1,
  'https://twitter.com/KeringGroup': 1,
  'https://www.facebook.com/keringgroup': 1,
  'https://www.linkedin.com/company/kering': 1,
  'https://www.youtube.com/channel/UCMDZYZibcyrf99iEa4CQJkA': 1,
  'https://www.tiktok.com/@keringgroup': 1,
  'https://www.weibo.com/keringcorp': 1,
  'http://i.youku.com/kering': 1},
 'internal': {'/it/maisons/kering-eyewear/': 2,
  '/it/il-gruppo/kering-for-women/women-in-motion/women-in-motion-e-le-donne-fotografe-al-festival-les-rencontres-d-arles-2023/': 2,
  '/it/maisons/moda-e-pelletteria/gucci/': 1,
  '/it/maisons/moda-e-pelletteria/saint-laurent/': 1,
  '/it/maisons/moda-e-pelletteria/bottega-veneta/': 1,
  '/it/maisons/moda-e-pelletteria/balenciaga/': 1,
  '/it/maisons/moda-e-pelletteria/alexander-mcqueen/': 1,
  '/it/maisons/moda-e-pelletteria/brioni/': 1,
  '/it/maisons/gioielli/boucheron/': 1,
  '/it/maisons/gioielli/pomellato/': 1,
  '/it/maisons/gioielli/dod

In [ ]:
# We can ESTIMATE the ratio based on a random sample of pages

# sample 5% of the site
sample_size = 0.05
# not more than n number of pages
max_n_samples = 10

In [ ]:
# table of pages to test
subset_of_sitemap_df = sitemap.sample(min(max_n_samples, round(sample_size*len(sitemap))))

# get domain
domain = urllib.parse.urlparse(homepage).netloc

# links per page
links_per_page = []

# get count of external links per pagex
for index, row in subset_of_sitemap_df.iterrows():

    page_url = row['loc']

    # count outbound links
    links = count_links( page_url, domain)

    # keep track of links per page
    if links.get('parseable'):
        external_links = len( links['external'] )
        links_per_page.append( external_links )

# create a dictionary to hold the data
data = {'page_url': [], 'external_links': []}

# fill the dictionary with data
for index, row in subset_of_sitemap_df.iterrows():
    page_url = row['loc']
    links = count_links( page_url, domain)
    if links.get('parseable'):
        external_links = len( links['external'] )
        data['page_url'].append(page_url)
        data['external_links'].append(external_links)

# convert dictionary to dataframe
df = pd.DataFrame(data)
df.to_excel('external_links_on_XML.xlsx', index=False)
df.head()

,page_url,external_links
0,https://www.kering.com/it/talenti/offerte-di-lavoro/gucci-client-advisor-tysons-galleria,8
1,https://www.kering.com/it/talenti/offerte-di-lavoro/europe/?page=62,8
2,https://www.kering.com/en/talent/job-offers/balenciaga-careers/?page=8,8
3,https://www.kering.com/cn/talent/job-offers/europe/?page=92,4
4,https://www.kering.com/fr/talent/offres-d-emploi/saint-laurent-stagiaire-assistant-gestion-de-production-hf,8


#Inspect the URL Structure

In [ ]:
sitemap_url = adv.url_to_df(df["loc"].fillna(''))
sitemap_url.head()

,url,scheme,netloc,path,query,fragment,dir_1,last_dir
0,https://yoast.com/seo-blog/,https,yoast.com,/seo-blog/,NaN,NaN,seo-blog,seo-blog
1,https://yoast.com/how-to-remove-www-from-your-url/,https,yoast.com,/how-to-remove-www-from-your-url/,NaN,NaN,how-to-remove-www-from-your-url,how-to-remove-www-from-your-url
2,https://yoast.com/google-analytics-segments-blog/,https,yoast.com,/google-analytics-segments-blog/,NaN,NaN,google-analytics-segments-blog,google-analytics-segments-blog
3,https://yoast.com/sequence-segments-google-analytics/,https,yoast.com,/sequence-segments-google-analytics/,NaN,NaN,sequence-segments-google-analytics,sequence-segments-google-analytics
4,https://yoast.com/secondary-dimensions-in-google-analytics/,https,yoast.com,/secondary-dimensions-in-google-analytics/,NaN,NaN,secondary-dimensions-in-google-analytics,secondary-dimensions-in-google-analytics


In [ ]:
adviz.value_counts_plus(sitemap_url['scheme'], size=20, name='Protocol')

,Protocol,count,cum. count,%,cum. %
1,https,"3,588","3,588",100.0%,100.0%


In [ ]:
adviz.value_counts_plus(sitemap_url['netloc'], size=20, name='Domain')

,Domain,count,cum. count,%,cum. %
1,www.bottegaveneta.com,"3,588","3,588",100.0%,100.0%


In [ ]:
adviz.value_counts_plus(sitemap_url['dir_1'], size=20, name='Market')

,Market,count,cum. count,%,cum. %
1,fr-fr,276,276,7.7%,7.7%
2,de-at,276,552,7.7%,15.4%
3,en-dk,276,828,7.7%,23.1%
4,en-gb,276,"1,104",7.7%,30.8%
5,en-no,276,"1,380",7.7%,38.5%
6,de-de,276,"1,656",7.7%,46.2%
7,en-es,276,"1,932",7.7%,53.8%
8,en-be,276,"2,208",7.7%,61.5%
9,en-nl,276,"2,484",7.7%,69.2%
10,en-pl,276,"2,760",7.7%,76.9%


In [ ]:
country = sitemap_url.groupby('dir_1').url.count().sort_values(ascending=False).reset_index()
country.rename(columns=
 {'dir_1':'country',
  'url': 'count'},
               inplace=True)

#plot a histogram
px.histogram(country,
             x='country',
             y='count',
             title='Ratio of Markets in XML sitemap',
             template='plotly_dark')

In [ ]:
adviz.value_counts_plus(sitemap_url['dir_2'], size=20, name='Main Directory')

,Main Directory,count,cum. count,%,cum. %
1,global,832,832,64.4%,64.4%
2,products,100,932,7.7%,72.2%
3,en,45,977,3.5%,75.7%
4,fr,38,"1,015",2.9%,78.6%
5,ku,27,"1,042",2.1%,80.7%
6,om,22,"1,064",1.7%,82.4%
7,sa,22,"1,086",1.7%,84.1%
8,bh,22,"1,108",1.7%,85.8%
9,uae,22,"1,130",1.7%,87.5%
10,KR,10,"1,140",0.8%,88.3%


In [ ]:
parent = sitemap_url.groupby('dir_2').url.count().sort_values(ascending=False).reset_index()
parent.rename(columns=
 {'dir_2':'parent category',
  'url': 'count'},
               inplace=True)

#plot a histogram

fig = px.histogram(
    parent.head(25),
    x='parent category',
    y='count',
    title='Main Parent Category pages in the XML sitemap',
    template='plotly_dark'
)

fig.update_layout(
    xaxis_tickangle=25
)

fig.show()


In [ ]:
#@title Orphan pages (URLs in sitemap, NOT crawled)
'''
we are using URLs found on the XML sitemap vs URLs found on the website
'''

import pandas as pd
from IPython.display import display_markdown

# Your existing code
in_sitemap_not_crawled = set(sitemap['loc'].str.strip('/')).difference(sitemap_url['url'].str.strip('/'))

# Calculate the percentage of pages not crawled
percentage_not_crawled = (len(in_sitemap_not_crawled) / len(sitemap)) * 100

# Create a DataFrame for the URLs not crawled
df_not_crawled = pd.DataFrame(in_sitemap_not_crawled, columns=['URL'])

# Save the DataFrame to an Excel file
output_file = 'not_crawled_urls.xlsx'
df_not_crawled.to_excel(output_file, index=False)

# Display the percentage
print(f"Percentage of pages in XML sitemap not crawled: {percentage_not_crawled:.2f}%")

# Show the URLs not crawled
for url in in_sitemap_not_crawled:
    display_markdown(f'[{url}]({url})', raw=True)

print(f"Output saved to {output_file}")






Percentage of pages in XML sitemap not crawled: 59.23%


[https://yoast.com/features/passive-voice-check](https://yoast.com/features/passive-voice-check)

[https://yoast.com/about-us/team/mazedul-islam-khan](https://yoast.com/about-us/team/mazedul-islam-khan)

[https://yoast.com/wordpress-core/beta-1-of-wordpress-5-9-postponed](https://yoast.com/wordpress-core/beta-1-of-wordpress-5-9-postponed)

[https://yoast.com/podcast/lily-ray](https://yoast.com/podcast/lily-ray)

...

[https://yoast.com/diversity-fund](https://yoast.com/diversity-fund)

[https://yoast.com/series/wordpress-for-beginners](https://yoast.com/series/wordpress-for-beginners)

[https://yoast.com/job-application-yoast-faq](https://yoast.com/job-application-yoast-faq)

[https://yoast.com/help/generate-set-google-maps-api-key](https://yoast.com/help/generate-set-google-maps-api-key)

Output saved to not_crawled_urls.xlsx


#What is the most frequent Country within a given XML sitemap?

In [ ]:
#@title Fetch Outlier URLs (or those pages that DO NOT reflect the purpose of the XML sitemap)

# Use boolean indexing to filter out URLs  that DO NOT target the U.S market
filtered_urls = sitemap[~sitemap['loc'].str.contains('/en_US/')]

# Create a new dataframe with the filtered rows
new_dataframe = pd.DataFrame(filtered_urls)

new_dataframe['loc'].to_excel('outlier_pages.xlsx',index=False)
new_dataframe['loc']

0                                                                                     https://www.kering.com/it
1                                                                             https://www.kering.com/it/finance
2              https://www.kering.com/it/il-gruppo/la-nostra-governance/consiglio-d-amministrazione/emma-watson
3                               https://www.kering.com/it/il-gruppo/la-nostra-governance/francois-henri-pinault
4        https://www.kering.com/it/il-gruppo/la-nostra-governance/consiglio-d-amministrazione/jean-pierre-denis
                                                          ...                                                  
18839                        https://www.kering.com/en/talent/job-offers/gucci-client-advisor-orlando-part-time
18840                                                 https://www.kering.com/en/talent/job-offers/gucci-cashier
18841                                    https://www.kering.com/en/talent/job-offers/gucci-operations-as

In [ ]:
#@title Most used words (uni-grams, bigrams, etc.) in the primary directory of the URLs included on the sitemap

(adv.word_frequency(
    sitemap_url['dir_2']
    .dropna()
    .str.replace('-', ' '),
    rm_words=['to', 'for', 'the', 'and', 'in', 'of', 'a', 'with', 'is'])
 .head(15)
 .style.format({'abs_freq': '{:,}'}))

,word,abs_freq
0,online.store,"4,907"
1,blog,109
2,accessories,83
3,storefinder,56
4,cameras,31
5,home,12
6,photo,8
7,studio,8
8,camcorders,6
9,consumables,5


In [ ]:
#@title Most used words (uni-grams, bigrams, etc.) in the SECONDARY directory of the URLs included on the sitemap
(adv.word_frequency(
    sitemap_url['dir_3']
    .dropna()
    .str.replace('-', ' '),
    rm_words=['to', 'for', 'the', 'and', 'in', 'of', 'a', 'with', 'is'])
 .head(15)
 .style.format({'abs_freq': '{:,}'}))

,word,abs_freq
0,recipe.html,730
1,recipes,377
2,products,370
3,product.html,197
4,nutrition,124
5,s,120
6,corn,110
7,kellogg,96
8,recipe,91
9,bran,91


## 🤖 ROBOTS.TXT

In [ ]:
#@title 5️⃣ Have a look at the Robots.txt
import requests
r = requests.get("https://www.jessops.com/robots.txt")
r.status_code

200

If the response status code is 200, it means there is a robots.txt file for the user-agent-based crawling control.



In [ ]:
#@title 6️⃣ Bulk audit Robots.txt of the URLs in the sitemap
sitemap_df_robotstxt_check = adv.robotstxt_test("https://yoast.com/robots.txt", urls=sitemap["loc"], user_agents=["*"])
sitemap_df_robotstxt_check["can_fetch"].value_counts()

True    2215
Name: can_fetch, dtype: int64

# 📓 SIDENOTES

**user_agents=["*"]** = we have performed the audit for all of the user-agents

**True** =  URLs are all crawlable.

**False** = some URLs are being disallowed

👇


---
If URLs are being disallowed, run the script below


In [ ]:
#@title Identify Disallowed URLs that are submitted in the XML sitemap
pd.set_option("display.max_colwidth",255)
sitemap_df_robotstxt_check[sitemap_df_robotstxt_check["can_fetch"] == False]

,robotstxt_url,user_agent,url_path,can_fetch


## STATUS CODE CHECK

In [ ]:
#@title Check URLs Status Code within the Sitemap
adv.crawl_headers(df["loc"], output_file="sitemap_df_header.jl")
df_headers = pd.read_json("sitemap_df_header.jl", lines=True)
df_headers["status"].value_counts()


200    903
Name: status, dtype: int64

1. ❌ Interrupt the parsing once the .jl file is loaded
2. **To check the URLs status code within the sitemap**, paste the .jl file onto the following line of code




In [ ]:
df_headers = pd.read_json('sitemap_df_header.jl', lines=True) # `lines=True` means that the rows from the JL output are being ordered as an array
df_headers.head()

,url,crawl_time,status,download_timeout,download_slot,download_latency,depth,protocol,body,resp_headers_content-length,resp_headers_date,resp_headers_content-type,resp_headers_vary,resp_headers_x-aspnet-version,resp_headers_cache-control,resp_headers_x-aspnetmvc-version,resp_headers_set-cookie,resp_headers_x-powered-by,resp_headers_cf-cache-status,resp_headers_server,resp_headers_cf-ray,resp_headers_content-encoding,request_headers_accept,request_headers_accept-language,request_headers_user-agent,request_headers_accept-encoding,redirect_times,redirect_ttl,redirect_urls,redirect_reasons,request_headers_cookie,resp_headers_expires,resp_headers_pragma,retry_times
0,https://www.jessops.com/accessories/camcorder-accessories/video-lights,2023-07-28 21:12:28,200,180,www.jessops.com,0.310501,0,HTTP/1.1,NaN,0,"Fri, 28 Jul 2023 21:12:28 GMT",text/html; charset=utf-8,Accept-Encoding,4.0.30319,private,5.2,"skin=; path=/; SameSite=None; Secure,VisitorCode=4a7c1e4f-aec8-4e58-b7f9-6a70f42bb97b; expires=Sun, 27-Aug-2023 21:1...",ASP.NET,DYNAMIC,cloudflare,7ee01eda3d837c67-LAX,gzip,"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",en,advertools/0.13.4,"gzip, deflate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.jessops.com/home-photo/slide-mounts-and-magazines,2023-07-28 21:12:29,200,180,www.jessops.com,0.752769,0,HTTP/1.1,NaN,0,"Fri, 28 Jul 2023 21:12:29 GMT",text/html; charset=utf-8,Accept-Encoding,4.0.30319,private,5.2,"skin=; path=/; SameSite=None; Secure,VisitorCode=9c83cfee-41db-4e71-a4d2-73637cb340da; expires=Sun, 27-Aug-2023 21:1...",ASP.NET,DYNAMIC,cloudflare,7ee01eda6e052efd-LAX,gzip,"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",en,advertools/0.13.4,"gzip, deflate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.jessops.com/accessories/tripods-and-supports,2023-07-28 21:12:29,200,180,www.jessops.com,0.755356,0,HTTP/1.1,NaN,0,"Fri, 28 Jul 2023 21:12:29 GMT",text/html; charset=utf-8,Accept-Encoding,4.0.30319,private,5.2,"skin=; path=/; SameSite=None; Secure,VisitorCode=f7a210e0-77e2-4e1b-9a03-b75b8a5df3f9; expires=Sun, 27-Aug-2023 21:1...",ASP.NET,DYNAMIC,cloudflare,7ee01eda6e690fb2-LAX,gzip,"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",en,advertools/0.13.4,"gzip, deflate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.jessops.com/studio/backgrounds,2023-07-28 21:12:29,200,180,www.jessops.com,0.895143,0,HTTP/1.1,NaN,0,"Fri, 28 Jul 2023 21:12:29 GMT",text/html; charset=utf-8,Accept-Encoding,4.0.30319,private,5.2,"skin=; path=/; SameSite=None; Secure,VisitorCode=01bd59cf-8ce8-4640-ab31-410809baa9ef; expires=Sun, 27-Aug-2023 21:1...",ASP.NET,DYNAMIC,cloudflare,7ee01eda595e08f7-LAX,gzip,"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",en,advertools/0.13.4,"gzip, deflate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.jessops.com/cameras/digital-slr-cameras/pentax,2023-07-28 21:12:29,200,180,www.jessops.com,0.598898,0,HTTP/1.1,NaN,0,"Fri, 28 Jul 2023 21:12:29 GMT",text/html; charset=utf-8,Accept-Encoding,4.0.30319,private,5.2,"skin=; path=/; SameSite=None; Secure,VisitorCode=1feb5643-f143-4534-9b05-5b6febfe58db; expires=Sun, 27-Aug-2023 21:1...",ASP.NET,DYNAMIC,cloudflare,7ee01edcaf8f7c67-LAX,gzip,"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",en,advertools/0.13.4,"gzip, deflate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#to save the crawl
df_headers.to_csv(r'iCloud Drive\Scrivania\url_status_code.csv', index = False, header=True)

In [ ]:
#@title Checkt whether the Sitemap houses any 404 URLs
df_headers[df_headers["status"] == 404]
#if nothing is returned, there are no 404s

,url,crawl_time,status,download_timeout,download_slot,download_latency,depth,protocol,body,resp_headers_Content-Length,resp_headers_Date,resp_headers_Content-Type,resp_headers_Cache-Control,resp_headers_Link,resp_headers_Strict-Transport-Security,resp_headers_Last-Modified,resp_headers_Cf-Cache-Status,resp_headers_Age,resp_headers_X-Content-Type-Options,resp_headers_Vary,resp_headers_Server,resp_headers_Cf-Ray,resp_headers_Content-Encoding,resp_headers_Alt-Svc,request_headers_Accept,request_headers_Accept-Language,request_headers_User-Agent,request_headers_Accept-Encoding,request_headers_Cookie


#Canonical check - find Canonicals that differ from Self







In [ ]:
#select only 200 canonicalised URLs
kg_cleaned = kg[(kg['status'] == 200) & (kg['canonical'] != kg['url'])]
#print results
kg_cleaned[kg_cleaned['canonical'].ne(kg['url'])][['url', 'canonical']]

NameError: name 'kg' is not defined

##Audit Meta Tag Robots



---
Even if a web page is not disallowed from robots.txt, it can still be disallowed from the HTML Meta Tags.

Thus, checking the HTML Meta Tags for better indexation and crawling is necessary.

Using the “custom selectors” is necessary to perform the HTML Meta Tag audit for the sitemap URLs.


---

📓 If "False" is the returned value, there are no URLs with **noindex|nofollow** attribute within the Robots command.


In [ ]:
adv.crawl(url_list=sitemap_url["loc"][:100], output_file="meta_command_audit.jl",

follow_links=False,

xpath_selectors= {"meta_command": "//meta[@name='robots']/@content"}, # to extract all the robots commands from the URLs from the sitemap.

custom_settings={"CLOSESPIDER_PAGECOUNT":100}) # we have set the crawling to 1000 URLs from hte sitemap

df_meta_check = pd.read_json("meta_command_audit.jl", lines=True)

df_meta_check["meta_command"].str.contains("nofollow|noindex", regex=True).value_counts()

KeyError: 'loc'

##⚠️ WARNING - If your GSC informs you about **indexing issues** stemming from a URL apparently displaying a **Index,Follow**, you should inspect the `<body>` section of the URL to assess whether a **noindex,follow** is in place

In [ ]:
#@title View the Meta Tag Robots applied on the Sitemap URLs
meta_tag_robots = df_meta_check[df_meta_check["meta_command"].str.contains("nofollow|noindex", regex=True) == True][["url", "meta_command"]]
#meta_tag_robots.to_csv('boh.csv',index=False)
meta_tag_robots

NameError: name 'df_meta_check' is not defined


#Check the duplicate URLs within the Sitemap URLs

In [ ]:
print(f'Original: {sitemap.shape}')
sitemap = sitemap.drop_duplicates(subset=['loc'])
print(f'After de-duplication: {sitemap.shape}')
sitemap.head()

Original: (2215, 22)
After de-duplication: (2215, 22)


,loc,lastmod,image,image_loc,sitemap,sitemap_size_mb,download_date,video,video_title,video_publication_date,video_description,video_player_loc,video_thumbnail_loc,video_duration,video_view_count,video_width,video_height,video_family_friendly,video_uploader,video_tag,video_content_loc,Pagination
0,https://yoast.com/jobs/,2024-01-08 09:39:56+00:00,NaN,NaN,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,https://yoast.com/jobs/retention-marketing-manager/,2023-12-22 12:18:57+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,https://yoast.com/jobs/seo-specialist/,2023-12-22 13:37:00+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/10/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,https://yoast.com/jobs/marketing-internship/,2024-01-08 09:39:56+00:00,\n\t\t\t,https://yoast.com/app/uploads/2023/12/image.png,https://yoast.com/yoast_jobs-sitemap.xml,0.003242,2024-01-27 18:37:01.199895+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,https://yoast.com/about-us/events/,2024-01-24 10:45:35+00:00,NaN,NaN,https://yoast.com/yoast_events-sitemap.xml,0.001593,2024-01-27 18:37:01.330218+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
duplicated_urls = sitemap['loc'].duplicated()
md(f'## Duplicated URLs: {duplicated_urls.sum():,} ({duplicated_urls.mean():.1%})')

## Duplicated URLs: 0 (0.0%)

In [ ]:
#@title Look for Duplicate URLs Within Sitemap Submissions
sitemap["loc"].duplicated().value_counts()

False    19227
Name: loc, dtype: int64

📓 **False = no duplicated URLs are uploaded in the sitemap**

❌ **True** = Duplicated URLs are caught in the Sitemap

👇

---
Should you have Duplicated URLs uploaded in your Sitemap, run the script below


In [ ]:
#@title ❌ How many duplicated URLs in the Sitemap?
pd.pivot_table(sitemap_df[sitemap_df["loc"].duplicated()==True], index="sitemap", values="loc", aggfunc="count").sort_values(by="loc", ascending=False)

,loc
sitemap,
https://seodepths.com/image-sitemap-1.xml,12


In [ ]:
#@title 💡Which URLs are caught as Duplicated in the Sitemap?
sitemap_df[sitemap_df["loc"].duplicated() == True]

,loc,lastmod,sitemap,sitemap_size_mb,download_date,image_loc,image_title,image_caption
16,https://seodepths.com/seo-news/google-pros-cons-annotations/,2022-07-12 15:27:16+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/office-freelancer-computer-business-38547.jpeg,office-freelancer-computer-business-38547,"Photo by Pixabay on <a href=""https://www.pexels.com/photo/coffee-apple-iphone-smartphone-38547/"" rel=""nofollow"">Pexels.com</a>"
22,https://seodepths.com/python-for-seo/detect-google-tag-rewriting-serpapi/,2022-07-13 09:26:52+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/pexels-photo-1181373.jpeg,pexels-photo-1181373,"Photo by Christina Morillo on <a href=""https://www.pexels.com/photo/person-using-macbook-pro-1181373/"" rel=""nofollow"">Pexels.com</a>"
23,https://seodepths.com/python-for-seo/sitemap-audit-python/,2022-07-14 11:57:10+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/pexels-photo-3935702-1.jpeg,pexels-photo-3935702-1,"Photo by Leah Kelley on <a href=""https://www.pexels.com/photo/woman-looking-at-the-map-3935702/"" rel=""nofollow"">Pexels.com</a>"
24,https://seodepths.com/python-for-seo/how-to-kick-off-entity-research-nlp-python/,2022-07-15 07:49:12+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/pexels-photo-2599244.jpeg,pexels-photo-2599244,"Photo by Alex Knight on <a href=""https://www.pexels.com/photo/high-angle-photo-of-robot-2599244/"" rel=""nofollow"">Pexels.com</a>"
25,https://seodepths.com/seo-news/how-nlp-nlu-can-affect-seo/,2022-07-14 16:22:56+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/things-not-keywords-1.webp,things-not-keywords-1,"Photo by Pixabay on <a href=""https://www.pexels.com/photo/advertising-alphabet-business-communication-270637/"" rel=""nofollow"">Pexels.com</a>"
26,https://seodepths.com/python-for-seo/define-seo-search-intent-with-python/,2022-07-16 15:59:06+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/commercial-intent.webp,commercial-intent,"Photo by Oleksandr Pidvalnyi on <a href=""https://www.pexels.com/photo/letters-on-the-wooden-blocks-9822732/"" rel=""nofollow"">Pexels.com</a>"
27,https://seodepths.com/python-for-seo/entity-and-sentiment-analysis-python/,2022-07-16 10:55:22+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/pexels-photo-4960630.jpeg,pexels-photo-4960630,"Photo by George Morina on <a href=""https://www.pexels.com/photo/macbook-pro-beside-black-iphone-4-on-table-4960630/"" rel=""nofollow"">Pexels.com</a>"
28,https://seodepths.com/about/,2022-07-20 08:48:11+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/C08D546B-8B62-49EA-8B1B-0F96D3786523-1.jpg,C08D546B-8B62-49EA-8B1B-0F96D3786523-1,"{""subsource"":""done_button"",""uid"":""29CF6EF8-00F2-4AAB-8805-5253DC0101EC_1597335319553"",""source"":""other"",""origin"":""gallery"",""source_sid"":""29CF6EF8-00F2-4AAB-8805-5253DC0101EC_1597335319571""}"
29,https://seodepths.com/seo-news/how-google-interprets-search-queries/,2022-07-18 15:18:59+00:00,https://seodepths.com/image-sitemap-1.xml,0.027044,2022-07-30 21:37:02.609921+00:00,https://seodepths.com/wp-content/uploads/2022/07/action-movies-starring-tom-cruise.webp,action-movies-starring-tom-cruise,"Photo by MART PRODUCTION on <a href=""https://www.pexels.com/photo/technology-computer-head-room-7088520/"" rel=""nofollow"">Pexels.com</a>"
30,https:/